# Mesurer avec un capteur Press-Hygro-Temp : Adafruit BME280#

Activité réalisée avec un capteur Adafruit BME280, une raspberry Pi3 fonctionnant avec une image Debian Stretch fournie par l'IFÉ ENS de Lyon, disposant des librairies INTEL mraa et upm.

Sur le site de Adafruit https://learn.adafruit.com/adafruit-bme280-humidity-barometric-pressure-temperature-sensor-breakout/pinouts on lit que 4 broches doivent être connectées au raspberryPi lorsqu'on travaille avec le bus I2C contient de nombreux capteurs cela devient possible.
* Vin - C'est la broche d'alimentation : elle peut-être relié à 3,3V ou 5V (Nous le relierons au 3,3V dans cet exercice). 
* GND - broche à relier à une broche ground du RasberryPi
* SCK - c'est la broche d'horloge du bus I2C à relier à la broche 5 (SCL)
* SDI - c'est la broche de données du bus I2C à relier à la broche 3 (SDA)

<span style='color:red'>ATTENTION : il ne faut pas brancher la broche Vin du capteur directement au 5V du RaspberryPi car le transfert des données se ferait aussi en 5V ce qui risquerait d'endommager le GPIO, par contre lorsque le bus `i2c` . </span> Nous verrons dans un autre exercice qu'il faut utiliser, pour les capteurs necessitant du 5V et ne fonctionnant pas en 3.3V, un [levelshifter (convertisseur de tension)](https://www.adafruit.com/product/757)

![Capteur BME280 branché sur le bus I2C d'un RaspberryPi](images/RaspberryPi_BME280L.jpg)

![Capteur BME280 branché sur le bus I2C d'un RaspberryPi](images/RaspberryPi_BME280Detail.jpg)


## Mesurer puis écrire la température en 6 lignes ##

Chaque ligne de texte ci-dessous est traduite dans le bloc suivant en une instruction dans un langage interprétable par la machine, à chaque alinéa correspond une instruction, la somme des instructions constitue un programme.

 * Demander l'utilisation des outils nécessaires, il sagit de deux bibliothèques de programmes de base qui permettent d'interagir et de piloter les capteurs que nous avons choisis. Le nom de ces bibliothèques est mraa et upm
 * parmi tous les pilotes disponibles nous devons utiliser le pilote pyupm_bmp280 (py pour python, upm nom de la librairie, _ séparateur, bme280 nom du capteur)
 * Déclarer le nom du capteur qu'on va utiliser (bme)
 * Mettre à jour le capteur (on utilise la commande préprogrammée dans le pilote "update")
 * Mesurer la température (on utilise la fonction préprogrammée dans le pilote "getTemperature()" on appelle la température tempBME)
 * Imprimer la température

In [15]:
import mraa, upm
from upm import pyupm_bmp280 as bmp
bme = bmp.BME280(0,0x77)
bme.update()
tempBME = bme.getTemperature()
print ("{0:.2f} °C".format(tempBME))

22.03 °C


## Afficher la température mesurée sur  l'afficheur
Le bloc de code suivant permet d'afficher la température mesurée  par le capteur BME280 sur l'afficheur vu précédemment

In [18]:
import mraa, time
from upm import pyupm_bmp280 as bmp
from upm import pyupm_jhd1313m1 as jdm
lcdAddress = 0x3E
rgbAddress = 0x62

bus1 = 0
bme = bmp.BME280(bus1)
lcd = jdm.Jhd1313m1(bus1,lcdAddress,rgbAddress)
bme.update()
tempBME = bme.getTemperature()
status = lcd.write("Temp : {0:.2f} C".format(tempBME))
time.sleep(10)
status = lcd.clear()
status = lcd.backlightOff()

### Exercice 
Améliorer l'affichage de la température (couleur défilement...)

## Afficher plusieurs mesures sur l'afficheur

Le bloc de code ci-dessous illustre les 3 mesures que  peut réaliser le capteur BME280, afficher ces mesures sur l'afficheur

In [19]:
import mraa, upm
from upm import pyupm_bmp280 as bmp
bus1 = 0
bme =bmp.BME280(bus1)
bme.update()
tempBME = bme.getTemperature()
pressBME = bme.getPressure() / 100.0
hygroBME = bme.getHumidity()
print ('Humidité : \tTempérature : \tPression :')
print ("{0:.2f} %\t\t{1:.2f} °C\t{2:.2f} hPa".format(hygroBME,tempBME,pressBME))

Humidité : 	Température : 	Pression :
36.35 %		21.81 °C	990.41 hPa


### Exercice
Afficher toutes les mesures effectuées par le capteur BME280

# Transposer l'affichage des codes ci-dessous sur l'afficheur

## Ajouter une fonction pour calculer la pression au niveau de la mer ##

La pression atmosphérique en un lieu dépend de son altitude. Pour pouvoir faire de la météorologie ou simplement comparer des mesures faites en différents endroits il faut partager une référence commune et le niveau de la mer est une solution simple et efficace. On convertit donc  notre mesure locale à la valeur qu'elle aurait  si nous étions au niveau de la mer. Il nous faut donc connaître notre altitude et importer des outils mathématiques. pour cela on ajoute la constante localAlt au programme et on importe la bibliothèque d'outils mathématiques de python.
Il est probable que nous ayons à réutiliser ce calcul  dans d'autres programmes et il serait malin de pouvoir réutiliser simplement les lignes de code que nous allons écrire. Plutôt que les noyer dans le programme nous allons _définir une fonction_ qui prendra un _paramètre_ (la pression mesurée m_Pressure) et qui nous _retournera_ la valeur à laquelle cette mesure correspond au niveau de la mer (sl_Pressure pour sea-level pressure). Le paramètre est une variable tout comme la valeur retournée.
La fonction est appelée ici convertSeaLevel() elle prend comme paramètre la variable mesurée par le capteur pressBME et fournit la variable décrivant  la pression au niveau de la mer seaLevelPress. On voit que les variables _locales_ utilisées par la fonction (m_Pressure, sl_Pressure) n'ont pas forcément le même nom que les variables _globales_ du programme principal (pressBME, seaLevelPress), cela permet de manipuler des noms de variables signifiants et cohérents avec leur contexte. Vous noterez que pouur souligner cette différence le nom des variables _locales à la fonction_ utilisent un système de séparation des mots (des "\_") différent de celui du programme principal ( le CamelCase).

In [5]:
import mraa, upm, math
from upm import pyupm_bmp280 as bmp
bus1 = 0
bme = bmp.BME280(bus1)
localAlt = 156.5
# Calcul de la pression au niveau de la mer
def convertSeaLevel (m_Pressure):
	sl_Pressure = m_Pressure / math.pow(1.0 - localAlt/44330, 5.255)
	return sl_Pressure
# fin du calcul
bme.update()
tempBME = bme.getTemperature()
pressBME = bme.getPressure() / 100.0
hygroBME = bme.getHumidity()
seaLevelPress = convertSeaLevel (pressBME)
print ('Humidité : \tTempérature : \tPression : \tPression(mer) : ')
print ("{0:.2f} %\t\t{1:.2f} °C\t{2:.2f} hPa\t{3:.2f} hPa".format(hygroBME,tempBME,pressBME,seaLevelPress))

Humidité : 	Température : 	Pression : 	Pression(mer) : 
31.29 %		32.55 °C	996.78 hPa	1015.48 hPa


## Faire une mesure toutes les 5 secondes ##

Lorsque l'on dispose d'un capteur et d'une alimentation suffisante il est bien sûr extrêmement intéressant de faire des mesures **en continu**. En fait on ne peut pas réellement mesurer de façon continue, on effectue des mesures les unes à la suite des autres en ménageant un temps (qui peut êetre très court) entre deux mesures pour transmettre le résultat et permettre au capteur d'être prêt pour la mesure suivante. La valeur du temps de pause est ici de 5 secondes valeur fournie à la fonction sleep (dors pendant....) de la librairie des programmes de gestion du temps (qui est incluse au début du programme).
Nous avons vu précédemment comment faire **une** mesure, nous venons de voir comment _ne rien faire_ pendant un temps donné, il nous faut donc enchaîner Mesure->Repos->Mesure->Repos-> c'est ce que l'on appelle une _boucle_. Pour le moment nous ne nous préoccupons pas de savoir comment arrêter les mesures et nous nous lançons dans une _boucle sans fin_; pour cela nous utilisons un _mot réservé_ : "True" (qui veut dire vrai) et nous utilisons l'instruction  "Tant que c'est vrai mesure puis attends 5s et recommence" comme nous déclarons que c'est vrai et que nous ne touchons pas cette valeur cela reste toujours vrai et on recommence toujours la série "mesure puis attends 5s". La _condition_ ayant pour valeur _"Tant que"_ que nous avons utilisée se traduit par _while_ et tout ce qui suit les ":" et est décalé de 4 caractères est répété à l'infini dans l'ordre des lignes.

In [ ]:
import mraa, upm, math, time
from upm import pyupm_bmp280 as bmp
bus1 = 0
pauseVal = 5
bme = bmp.BME280(bus1)
localAlt = 156.5
# Calcul de la pression au niveau de la mer
def convertSeaLevel (m_Pressure):
	sl_Pressure = m_Pressure / math.pow(1.0 - localAlt/44330, 5.255)
	return sl_Pressure
# fin du calcul
print ('Humidité : \tTempérature : \tPression : \tPression(mer) : ')
while True :
    bme.update()
    tempBME = bme.getTemperature()
    pressBME = bme.getPressure() / 100.0
    hygroBME = bme.getHumidity()
    seaLevelPress = convertSeaLevel (pressBME)
    print ("{0:.2f} %\t\t{1:.2f} °C\t{2:.2f} hPa\t{3:.2f} hPa".format(hygroBME,tempBME,pressBME,seaLevelPress))
    time.sleep (pauseVal)

Humidité : 	Température : 	Pression : 	Pression(mer) : 
36.67 %		21.69 °C	990.49 hPa	1009.07 hPa
36.53 %		21.68 °C	990.47 hPa	1009.05 hPa
36.57 %		21.68 °C	990.49 hPa	1009.07 hPa
36.41 %		21.68 °C	990.47 hPa	1009.05 hPa
36.58 %		21.68 °C	990.47 hPa	1009.05 hPa
36.59 %		21.69 °C	990.47 hPa	1009.05 hPa
